# Обновление `portfolio_whitelist.txt`

Этот ноутбук помогает обновлять список инструментов для портфеля: загружаем вселенную акций MOEX, анализируем фьючерсы, вручную выбираем нужные и сохраняем в `portfolio_whitelist.txt`.

## 1. Импорт зависимостей и настройка путей

In [1]:


import pandas as pd
import ipywidgets as widgets
from IPython.display import display



import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
while PROJECT_ROOT != PROJECT_ROOT.parent and not (PROJECT_ROOT / "src" / "tvr_service").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

if not (PROJECT_ROOT / "src" / "tvr_service").exists():
    raise RuntimeError("Не удалось найти каталог src/tvr_service")

SRC_DIR = PROJECT_ROOT / "src"
sys.path.insert(0, str(SRC_DIR))


from tvr_service.pipeline import (
    get_top_futures,
    load_share_universe,
)

DEFAULT_WHITELIST_PATH = PROJECT_ROOT / 'src' / 'tvr_service' / 'pipeline' / 'data' / 'portfolio_whitelist.txt'
DEFAULT_SHARE_UNIVERSE_PATH = PROJECT_ROOT / 'src' / 'tvr_service' / 'pipeline' / 'data' / 'share_universe.csv'


## 2. Параметры выборки

Задаём дату, сортировку, размер выборки и пути к файлам. При необходимости меняйте и перезапускайте ячейки.

In [2]:
DATE = '2025-09-25'          # дата для расчёта
SORT_BY = 'VALUE'               # поле для сортировки: VALUE / VOLUME / NUMTRADES
TOP_N = None                      # сколько инструментов оставить для ручного отбора (None = все)
FETCH_LIMIT = 100              # лимит строк при загрузке истории
WHITELIST_PATH = DEFAULT_WHITELIST_PATH
SHARE_UNIVERSE_PATH = DEFAULT_SHARE_UNIVERSE_PATH
SHARES_REFRESH = False          # True, чтобы перекачать вселенную акций заново


## 3. Загрузка вселенной акций

In [3]:
share_universe = load_share_universe(
    share_universe_path=SHARE_UNIVERSE_PATH,
    use_cache=True,
    refresh=SHARES_REFRESH,
    save_to_cache=True,
)
print(f'Всего акций в базе: {len(share_universe)}')
share_universe.head()

Всего акций в базе: 258


,SECID,SECTOR
0,ABIO,nan
1,ABRD,nan
2,AFKS,nan
3,AFLT,nan
4,ALRS,nan


## 4. Загрузка всех фьючерсов (без фильтров)

In [6]:
all_futures = get_top_futures(
    date=DATE,
    sort_by=SORT_BY,
    top_n=None,
    limit=FETCH_LIMIT,
    only_equities=False,
    share_universe_path=SHARE_UNIVERSE_PATH,
    shares_use_cache=True,
)
print(f'Всего фьючерсов после агрегации: {len(all_futures)}')
# all_futures[['SECID', 'SHORTNAME', 'base_code', 'VALUE', 'VOLUME', 'NUMTRADES']].head()
all_futures[['base_code', 'VALUE', 'VOLUME', 'NUMTRADES']].head()

Всего фьючерсов после агрегации: 414


,base_code,VALUE,VOLUME,NUMTRADES
0,GOLD,9.220779e+10,295325.0,85717
1,SI,6.533140e+10,752128.0,130091
2,CNY,5.662111e+10,4651341.0,130745
3,NG,5.138061e+10,2009391.0,220276
4,MIX,4.692923e+10,167153.0,76366


In [6]:
# checkboxes = []
# for idx, row in shortlist.iterrows():
#     label = f"{row['base_code']} | {row['SHORTNAME']} | VALUE={row['VALUE']:.0f} | VOLUME={row['VOLUME']:.0f} | TRADES={row['NUMTRADES']}"
#     cb = widgets.Checkbox(value=False, description=label, indent=False)
#     checkboxes.append(cb)
# checkbox_box = widgets.VBox(checkboxes)
# checkbox_box

## 5. Фильтр по акциям

In [ ]:
equity_futures = set(share_universe["SECID"].astype(str).str.upper())
filtered = all_futures[
    all_futures["base_code"].astype(str).str.upper().isin(equity_codes)
].copy()
equity_futures.head()

In [5]:
equity_futures = get_top_futures(
    date=DATE,
    sort_by=SORT_BY,
    top_n=None,
    limit=FETCH_LIMIT,
    only_equities=True,
    share_universe=share_universe,
    share_universe_path=SHARE_UNIVERSE_PATH,
    shares_use_cache=True,
)
print(f'Фьючерсов на акции: {len(equity_futures)}')
equity_futures = equity_futures[['SECID', 'SHORTNAME', 'base_code', 'VALUE', 'VOLUME', 'NUMTRADES']]
equity_futures.head()

Фьючерсов на акции: 96


,SECID,SHORTNAME,base_code,VALUE,VOLUME,NUMTRADES
0,VBZ5,VTBR-12.25,VTBR,2.877429e+09,388159.0,31676
1,LKZ5,LKOH-12.25,LKOH,6.722578e+08,10653.0,4044
2,YDZ5,YDEX-12.25,YDEX,3.435369e+08,82062.0,5846
3,RNZ5,ROSN-12.25,ROSN,3.413223e+08,7657.0,2798
4,GKZ5,GMKN-12.25,GMKN,2.762941e+08,222088.0,6056


## 6. Сортировка и выборка ТОП N

In [6]:
sorted_futures = equity_futures.sort_values(by=SORT_BY, ascending=False).reset_index(drop=True)
if TOP_N is not None:
    shortlist = sorted_futures.head(TOP_N).reset_index(drop=True)
else:
    shortlist = sorted_futures.copy()
print(f'Инструментов в коротком списке: {len(shortlist)}')
shortlist

Инструментов в коротком списке: 96


,SECID,SHORTNAME,base_code,VALUE,VOLUME,NUMTRADES
0,VBZ5,VTBR-12.25,VTBR,2.877429e+09,388159.0,31676
1,LKZ5,LKOH-12.25,LKOH,6.722578e+08,10653.0,4044
2,YDZ5,YDEX-12.25,YDEX,3.435369e+08,82062.0,5846
3,RNZ5,ROSN-12.25,ROSN,3.413223e+08,7657.0,2798
4,GKZ5,GMKN-12.25,GMKN,2.762941e+08,222088.0,6056
...,...,...,...,...,...,...
91,CMH6,CBOM-3.26,CBOM,1.543000e+04,2.0,2
92,KMH6,KMAZ-3.26,KMAZ,7.705000e+03,8.0,3
93,SEH6,SPBE-3.26,SPBE,5.152000e+03,2.0,2
94,BNH6,BANE-3.26,BANE,3.479000e+03,2.0,2


## 7. Ручной отбор через чек-боксы

Отметьте галочками инструменты, которые нужно внести в whitelist. По умолчанию все включены.

In [7]:
checkboxes = []
for idx, row in all_futures.iterrows():
    label = f"{row['base_code']} | VALUE={row['VALUE']:.0f} | VOLUME={row['VOLUME']:.0f} | TRADES={row['NUMTRADES']}"
    cb = widgets.Checkbox(value=False, description=label, indent=False)
    checkboxes.append(cb)
checkbox_box = widgets.VBox(checkboxes)
checkbox_box

## 8. Просмотр выбранных инструментов и сохранение

In [ ]:
def _selected_indices():
    return [i for i, cb in enumerate(checkboxes) if cb.value]

preview_output = widgets.Output()
save_output = widgets.Output()

preview_button = widgets.Button(description='Показать выбранные', button_style='info')
save_button = widgets.Button(description='Сохранить в whitelist', button_style='success')

list_box = widgets.VBox([preview_button, preview_output, save_button, save_output])

def _on_preview(_):
    with preview_output:
        preview_output.clear_output()
        idxs = _selected_indices()
        if not idxs:
            print('Ничего не выбрано.')
            return
        selected = all_futures.iloc[idxs]
        print(f'Выбрано {len(selected)} инструментов:')
        display(selected)

def _on_save(_):
    with save_output:
        save_output.clear_output()
        idxs = _selected_indices()
        if not idxs:
            print('Нечего сохранять — отметьте хотя бы один инструмент.')
            return
        selected_codes = all_futures.iloc[idxs]['base_code'].astype(str).str.upper().tolist()
        unique_codes = sorted(set(selected_codes))
        WHITELIST_PATH.parent.mkdir(parents=True, exist_ok=True)
        WHITELIST_PATH.write_text('\n'.join(unique_codes), encoding='utf-8')
        print(f'Сохранено {len(unique_codes)} тикеров в {WHITELIST_PATH}')
        print('Для проверки содержание файла можно открыть в отдельной вкладке.')

preview_button.on_click(_on_preview)
save_button.on_click(_on_save)
list_box